# Analysis of datasets - source: MagicDraw model

Mission: verify the integrity of a system model

Objectives: 
- use pattern matching to detect the inconsistencies,
- use graph analysis to display correction suggestions,
- allocate the correction of the system model to a user, selected as being familiar with the CAESAR WorkBench v1.23,
- assume that the test is finished when the graph analysis returns that the input model complies with the 3 different rules.

Note: the RDF dataset shall be 
- renamed: *.trig
- available through a query endpoint

In [253]:
# import necessary libraries 
import pandas as pd
from pandas.io.json import json_normalize
from SPARQLWrapper import SPARQLWrapper, JSON

import ipywidgets as widgets
from ipywidgets import Layout
from IPython.display import Javascript

from urllib.request import Request, urlopen
from urllib.error import URLError, HTTPError

import networkx as nx
import snap

In [254]:
# definition of a function: sparql query
def query_fusekiData(sparql_query, sparql_service_url):
    """
    Query the endpoint with the given query string and return the results as a pandas Dataframe.
    """
    # create the connection to the endpoint
    sparql = SPARQLWrapper(sparql_service_url)
    
    sparql.setQuery(sparql_query)
    sparql.setReturnFormat(JSON)

    # ask for the result
    result = sparql.query().convert()
    return json_normalize(result["results"]["bindings"])

In [255]:
# definition of ipywidget 'Text' to prompt the  endpoint to query
layout = widgets.Layout(width='auto', height='40px') #set width and height of the different widgets
endpoint=widgets.Text(
    value='http://localhost:3030/MDv20191202',
    placeholder='http://localhost:3030/',
    description='Endpoint: ',
    layout = layout,
    disabled=False
)
display(endpoint)

Text(value='http://localhost:3030/MDv20191202', description='Endpoint: ', layout=Layout(height='40px', width='…

In [256]:
# definition of ipywidget 'Button' to take into account the endpoint value prompted
output = widgets.Output()
def on_button_clicked(b):
    with output:
        display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.ncells())'))
        
b = widgets.Button(
    description='Analyze dataset', 
    layout = layout,
    disabled=False)
display(b)
b.on_click(on_button_clicked)

Button(description='Analyze dataset', layout=Layout(height='40px', width='auto'), style=ButtonStyle())

In [257]:
# verification of the endpoint URL
req = Request(endpoint.value)
try:
    response = urlopen(req)
except HTTPError as e:
    print('Error - Endpoint: ' +endpoint.value + ' not found. Verify URL. (Error code: ', e.code, ')')
    raise SystemExit
except URLError as e:
    print('Error - Server not reached. (Reason: ', e.reason, ')')
    raise SystemExit
else:
    print ('Endpoint: ' +endpoint.value + ' found. Processing results now...')

Endpoint: http://localhost:3030/MDv20191202 found. Processing results now...


# Rule #1: Structural decomposition graph shall be directed, acyclic, connected and rooted

In [258]:
# composition sparql query
query = """
PREFIX owl:   <http://www.w3.org/2002/07/owl#>
PREFIX rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX base:  <http://imce.jpl.nasa.gov/foundation/base/base#>

SELECT DISTINCT ?physicalComponent ?isContainedIn
FROM <urn:x-arq:UnionGraph>
WHERE
{
  ?a a owl:Class ;
     rdfs:label ?b ;
     rdfs:subClassOf <http://firesat.jpl.nasa.gov/user-model/generated/md/profileExt/firesat-extensions_ID__18_5_3_8c20287_1563932039371_543598_24020#PhysicalComponent_ID__18_5_3_8c20287_1563932086646_143892_24055>.

  ?c a owl:Class ;
     rdfs:label ?physicalComponent ;
     rdfs:subClassOf <http://firesat.jpl.nasa.gov/user-model/generated/md/profileExt/firesat-extensions_ID__18_5_3_8c20287_1563932039371_543598_24020#PhysicalComponent_ID__18_5_3_8c20287_1563932086646_143892_24055>.


  optional {?a rdfs:subClassOf [a owl:Restriction ;
               owl:onProperty base:aggregates ;
               owl:someValuesFrom ?c] }.

  optional {?e rdfs:subClassOf [a owl:Restriction ;
               owl:onProperty base:aggregates ;
               owl:someValuesFrom ?c ].
    ?e rdfs:label ?isContainedIn }.
}
"""
if query_fusekiData(query, endpoint.value).empty:
    print('Error - The pattern matching query returned no results. Verify OWL data.')
    raise SystemExit
else:
    result_table = query_fusekiData(query, endpoint.value)

In [259]:
if 'physicalComponent.value' not in result_table.columns:
    print('Error - No Physical Component found. Verify OWL data.')
    raise SystemExit
else:
    obj = result_table[['physicalComponent.value', 'isContainedIn.value']]
    obj = obj.rename(columns = lambda col: col.replace(".value", ""))

In [260]:
# from dataframe to graph
objGraph = nx.from_pandas_edgelist(obj, 'physicalComponent', 'isContainedIn', create_using=nx.DiGraph())

In [261]:
# is graph directed
print("Graph is directed:", nx.is_directed(objGraph))

Graph is directed: True


In [262]:
# is graph acyclic
print("Graph is acyclic:", nx.is_directed_acyclic_graph(objGraph))
if nx.is_directed_acyclic_graph(objGraph) != True:
    print("Number of cycles:", len(list(nx.simple_cycles(objGraph))))
    print("Cycles found:")
    for el in list(nx.simple_cycles(objGraph)):
        print('- ', el)
    print("Among them, number of self-loop edges:", objGraph.number_of_selfloops())
    print("Nodes presenting a self-loop edge:" )
    for el in objGraph.nodes_with_selfloops():
        print('- ', el)
    print("\nSuggestion: Correct the graph so that it becomes acyclic.")

Graph is acyclic: True


In [263]:
# is graph connected
print("Graph is connected:", nx.is_weakly_connected(objGraph))
if nx.is_weakly_connected(objGraph) != True:
    print("Number of isolated nodes:", len(list(nx.isolates(objGraph))))
    print("Isolated nodes:")
    for el in list(nx.isolates(objGraph)):
        print('- ', el)
    print("\nSuggestion: Connect the isolated nodes.")

Graph is connected: True


In [264]:
# is graph rooted
obj = pd.DataFrame(objGraph.out_degree())
obj = obj[obj[1]==0]
print("Graph is rooted:", obj.shape[0] == 1)
if obj.shape[0] == 0:
    print("No roots found.")
    print("\nSuggestion: Correct the graph so that it presents one and only one root.")
if obj.shape[0] > 1:
    print("Number of roots found:", obj.shape[0])
    print("Roots found:")
    for el in obj[0]:
        print('-', el)
    print("\nSuggestion: Correct the graph so that it presents one and only one root.")

Graph is rooted: True


# Rule #2: Each Physical Component shall be contained in exactly one Subsystem

In [265]:
# aggregation sparql query
query = """
PREFIX owl:   <http://www.w3.org/2002/07/owl#>
PREFIX rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX base:  <http://imce.jpl.nasa.gov/foundation/base/base#>

SELECT DISTINCT ?physicalComponent ?subSystem
FROM <urn:x-arq:UnionGraph>
WHERE
{
  ?a a owl:Class ;
     rdfs:label ?b ;
     rdfs:subClassOf <http://firesat.jpl.nasa.gov/user-model/generated/md/profileExt/firesat-extensions_ID__18_5_3_8c20287_1563932039371_543598_24020#PhysicalComponent_ID__18_5_3_8c20287_1563932086646_143892_24055>.

  ?c a owl:Class ;
     rdfs:label ?physicalComponent ;
     rdfs:subClassOf <http://firesat.jpl.nasa.gov/user-model/generated/md/profileExt/firesat-extensions_ID__18_5_3_8c20287_1563932039371_543598_24020#PhysicalComponent_ID__18_5_3_8c20287_1563932086646_143892_24055>.


  optional {?a rdfs:subClassOf [a owl:Restriction ;
               owl:onProperty base:contains ;
               owl:someValuesFrom ?c] }.

  optional {?e rdfs:subClassOf [a owl:Restriction ;
               owl:onProperty base:contains ;
               owl:someValuesFrom ?c ].
    ?e rdfs:label ?subSystem }.
}
"""
if query_fusekiData(query, endpoint.value).empty:
    print('Error - The pattern matching query returned no results. Verify OWL data.')
    raise SystemExit
else:
    result_table = query_fusekiData(query, endpoint.value)

In [266]:
if 'subSystem.value' not in result_table.columns:
    print('Error - No Subsystems found. Verify OWL data.')
    raise SystemExit
else:
    obj = result_table[['physicalComponent.value', 'subSystem.value']]
    obj = obj.rename(columns = lambda col: col.replace(".value", ""))

In [267]:
# from dataframe to graph
objGraph = nx.from_pandas_edgelist(obj, 'physicalComponent', 'subSystem', create_using=nx.DiGraph())

### a/ Is there Physical Component not contained in any Subystem?

In [268]:
sol_a = obj.subSystem.isnull().sum() # select rows based on condition 

print('Number of physical component not aggregated in any subsystem: '+str(sol_a))
sol_a = obj[obj['subSystem'].isnull()] 
sol_a = sol_a['physicalComponent'].tolist()

if not sol_a:
    print('The model complies with the rule. No action needs to be performed.')
else:
    print('\nList of physical component currently not aggregated in any subSystem: ')
    for el in sol_a:
        print ("- " +el)
    print("\nSuggestion: Connect each of these physical components to one subSystem.")

Number of physical component not aggregated in any subsystem: 0
The model complies with the rule. No action needs to be performed.


### b/ Is there any Physical Component contained in more than one Subsystem?

In [269]:
sol_b = obj.groupby(["physicalComponent"]).size()
sol_b = sol_b.to_frame("subSystem_count")

sol_b = sol_b[sol_b['subSystem_count'] > 1] # select rows based on condition 
print('Number of physical components aggregated in more than one Subsystem: '+str(sol_b.shape[0]))

if sol_b.empty:
    print('The model complies with the rule. No action needs to be performed.')
else:
    print('\nList of physical components currently aggregated in more than one SubSystem: ')
    for i in range(sol_b.shape[0]):
        print('- Physical component ' +list(sol_b.index)[i] +' is aggregated in ' +str(list(sol_b.subSystem_count)[i]) +' different SubSystems: ')
        for j in range((obj[obj['physicalComponent'] == list(sol_b.index)[i]]).shape[0]):
            print('  o ' +list(obj[obj['physicalComponent'] == list(sol_b.index)[i]].subSystem)[j])
    print("\nSuggestion: Look for these Physical Components and delete the extra aggregations.")

Number of physical components aggregated in more than one Subsystem: 0
The model complies with the rule. No action needs to be performed.


# Rule #3: Each Subsystem shall be supplied by exactly one WorkPackage

In [270]:
# workpackage sparql query
query = """
PREFIX project: <http://imce.jpl.nasa.gov/foundation/project/project#>
PREFIX owl:   <http://www.w3.org/2002/07/owl#>
PREFIX rdfs:  <http://www.w3.org/2000/01/rdf-schema#>

SELECT DISTINCT ?subsystem ?workPackage

FROM <urn:x-arq:UnionGraph>
WHERE
{
  ?a rdfs:subClassOf <http://firesat.jpl.nasa.gov/user-model/generated/md/profileExt/firesat-extensions_ID__18_5_3_8c20287_1563932039371_543598_24020#Subsystem_ID__18_5_3_8c20287_1572979614280_430250_25878>;
     rdfs:label ?subsystem.
  
  optional {
    ?b rdfs:subClassOf  [ a owl:Restriction ; owl:onProperty project:supplies ; owl:someValuesFrom ?a ] ;
                                                                                rdfs:label ?workPackage. }  
}
"""
if query_fusekiData(query, endpoint.value).empty:
    print('Error - The pattern matching query returned no results. Verify OWL data.')
    raise SystemExit
else:
    result_table = query_fusekiData(query, endpoint.value)

In [271]:
if 'subsystem.value' not in result_table.columns:
    print('Error - No Subsystem found. Verify OWL data.')
    raise SystemExit
else:
    obj = result_table[["subsystem.value", "workPackage.value"]]
    obj = obj.rename(columns = lambda col: col.replace(".value", ""))

### a/ Is there subsytems not supplied by any WorkPackage?

In [272]:
sol_a = obj.workPackage.isnull().sum() # select rows based on condition 

print('Number of Subsystems not supplied by any WorkPackage: '+str(sol_a))
sol_a = obj[obj['workPackage'].isnull()] 
sol_a = sol_a['subsystem'].tolist()

if not sol_a:
    print('The model complies with the rule. No action needs to be performed.')
else:
    print('\nList of Subsystems currently not supplied by any WorkPackage: ')
    for el in sol_a:
        print ("- " +el)
    print("\nSuggestion: Connect each of these Subsystems to one WorkPackage.")

Number of Subsystems not supplied by any WorkPackage: 0
The model complies with the rule. No action needs to be performed.


### b/ Is there subystems supplied by more than one WorkPackage?

In [273]:
sol_b = obj.groupby(["subsystem"]).size()
sol_b = sol_b.to_frame("workPackage_count")

sol_b = sol_b[sol_b['workPackage_count'] > 1] # select rows based on condition 
print('Number of Subsystems supplied by more than one WorkPackage: '+str(sol_b.shape[0]))

if sol_b.empty:
    print('The model complies with the rule. No action needs to be performed.')
else:
    print('\nList of subsystems currently supplied by more than one WorkPackage: ')
    for i in range(sol_b.shape[0]):
        print('- Subsystem ' +list(sol_b.index)[i] +' has relationship(s) with ' +str(list(sol_b.workPackage_count)[i]) +' different WorkPackages: ')
        for j in range((obj[obj['subsystem'] == list(sol_b.index)[i]]).shape[0]):
            print('  o ' +list(obj[obj['subsystem'] == list(sol_b.index)[i]].workPackage)[j])
    print("\nSuggestion: Look for these Subsytems and delete the extra relationships.")

Number of Subsystems supplied by more than one WorkPackage: 0
The model complies with the rule. No action needs to be performed.


# Rule #4: Each Physical Component shall perform at least one Function

In [274]:
# assemblies perform functions sparql query
query = """
PREFIX owl:   <http://www.w3.org/2002/07/owl#>
PREFIX rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX base:  <http://imce.jpl.nasa.gov/foundation/base/base#>
PREFIX mission: <http://imce.jpl.nasa.gov/foundation/mission/mission#>

SELECT DISTINCT ?physicalComponent ?isContainedIn ?function
FROM <urn:x-arq:UnionGraph>
WHERE
{
  ?a a owl:Class ;
     rdfs:label ?b ;
     rdfs:subClassOf <http://firesat.jpl.nasa.gov/user-model/generated/md/profileExt/firesat-extensions_ID__18_5_3_8c20287_1563932039371_543598_24020#PhysicalComponent_ID__18_5_3_8c20287_1563932086646_143892_24055>.

  ?c a owl:Class ;
     rdfs:label ?physicalComponent ;
     rdfs:subClassOf <http://firesat.jpl.nasa.gov/user-model/generated/md/profileExt/firesat-extensions_ID__18_5_3_8c20287_1563932039371_543598_24020#PhysicalComponent_ID__18_5_3_8c20287_1563932086646_143892_24055>.

  optional {?a rdfs:subClassOf [a owl:Restriction ;
               owl:onProperty base:aggregates ;
               owl:someValuesFrom ?c] }.

  optional {?e rdfs:subClassOf [a owl:Restriction ;
               owl:onProperty base:aggregates ;
               owl:someValuesFrom ?c ].
    ?e rdfs:label ?isContainedIn }.
    
  optional {
    ?c rdfs:subClassOf  [ a owl:Restriction ; owl:onProperty mission:performs ; owl:someValuesFrom ?g ].
    ?g rdfs:label ?function
    }
}
"""
if query_fusekiData(query, endpoint.value).empty:
    print('Error - The pattern matching query returned no results. Verify OWL data.')
    raise SystemExit
else:
    result_table = query_fusekiData(query, endpoint.value)

In [275]:
if 'physicalComponent.value' not in result_table.columns:
    print('Error - No Physical Component found. Verify OWL data.')
    raise SystemExit
else:
    obj = result_table[['physicalComponent.value', 'isContainedIn.value', 'function.value']]
    obj = obj.rename(columns = lambda col: col.replace(".value", ""))

In [276]:
# List the parent assemblies to remove the implied relationships from the analysis 
obj_a = obj[['isContainedIn', 'function']]
obj_a = obj_a.drop_duplicates(subset = 'isContainedIn')
obj_a = obj_a.dropna(axis = 0, subset = ['isContainedIn'])
obj_a = obj_a.rename(columns={'isContainedIn':'physicalComponent'})

# List all the assemblies
obj_b = obj[['physicalComponent', 'function']]
obj_b = obj_b.drop_duplicates(subset = 'physicalComponent')

# List the assemblies that aggregate no childs
obj = pd.concat([obj_a, obj_b])
obj = obj.drop_duplicates(subset = 'physicalComponent', keep = False)

In [277]:
sol = obj.function.isnull().sum() # select rows based on condition 

print('Number of Physical Components not performing any Function: '+str(sol))
sol = obj[obj['function'].isnull()] 
sol = sol['physicalComponent'].tolist()

if not sol:
    print('The model complies with the rule. No action needs to be performed.')
else:
    print('\nList of Subsystems currently not supplied by any WorkPackage: ')
    for el in sol:
        print ("- " +el)
    print("\nSuggestion: Connect each of these Subsystems to one WorkPackage.")

Number of Physical Components not performing any Function: 0
The model complies with the rule. No action needs to be performed.


# Rule #5: Each Physical Component shall satisfy at least one non functional Requirement

# Rule #6: Each Function shall be performed by at least one Physical Component

# Rule #7: Each Requirement shall specify at least one Physical Component

# Rule #8: Each functional Requirement shall specify how the Physical Components perform the function